In [1]:
import tensorflow as tf

2024-05-12 23:10:25.514142: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 23:10:25.719504: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 23:10:25.720923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 23:10:26.607346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [25]:
from src.brain_tumor_classification.utils import *
from src.brain_tumor_classification.exception import Custom_Exception

In [22]:
from tensorflow.keras.models import save_model

In [13]:
os.getcwdb()

b'/home/googlyji/ml_projects/dl_projects/brain_tumor/brain-tumor-classification'

In [10]:
os.chdir("../")

In [12]:
import os 

In [36]:
from src.brain_tumor_classification.exception import Custom_Exception

In [ ]:
from tensorflow.keras.applications import saved_model

In [14]:
from src.brain_tumor_classification.constants import *

In [2]:
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,Flatten,Dropout
from tensorflow.keras.models import Model,Sequential

In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from dataclasses import dataclass
from pathlib import Path

In [34]:
from src.brain_tumor_classification.utils.common import *

In [44]:
@dataclass
class Base_model:
    root_dir: Path
    model_weights: Path
    updated_model_weights:Path
    Image_size: list
    Batch_size: int
    weights: str
    epochs: int
    classes: int
    include_top:bool


In [45]:
# configuration manger
class Configuration_manager:
    def __init__(self,config=CONFIG_PATH,params=PARAMS_PATH):
        self.config=read_yaml(config)
        self.params=read_yaml(params)
        
        create_directories([self.config.artifacts_root])

    def base_model_config(self)->Base_model:
        config=self.config.base_model

        base_model_config=Base_model(root_dir=config.root_dir,model_weights=config.model_weights,updated_model_weights=config.updated_model_weights,
                                    Image_size=self.params.Image_size, 
                                    Batch_size=self.params.Batch_size,
                                    weights=self.params.weights,
                                    epochs=self.params.epochs,
                                    classes=self.params.classes,
                                    include_top=self.params.include_top)
        

        return base_model_config
        



        


In [61]:
# components 
class Prepare_base_model:
    
    def __init__(self,config1:Base_model):
        self.config=config1

    def prepare_base_model(self):
        self.base_mod=tf.keras.applications.vgg16.VGG16(input_shape=self.config.Image_size,include_top=self.config.include_top,weights=self.config.weights)

        self.base_mod.summary()

        save_model(filepath=self.config.model_weights,model=self.base_mod)
    


    def prepare_full_model(self,model):
        for i in model.layers:
            i.trainable=False   ## it will preserve the initail weights of the model 

        x=model.output 
        x=tf.keras.layers.GlobalAveragePooling2D()(x)
        x=tf.keras.layers.Dense(256,activation="relu")(x)
        prediction=tf.keras.layers.Dense(4,activation="softmax")

        model=Model(inputs=model.input,outputs=prediction)

        model.compile(optimizer=tf.keras.optimizers.Adam,loss=tf.keras.losses.CategoricalCrossentropy,metrics=["accuracy"])

        model.summary()

        save_model(filepath=self.config.updated_model_weights,model=model)

        return model 
    
    
    








        

In [62]:
try:
    config=Configuration_manager()
    config=config.base_model_config()
    a=Prepare_base_model(config1=config)
    a.prepare_base_model()
    
except Exception as e:
    raise(Custom_Exception(e,sys))

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

/tmp/ipykernel_96823/673948538.py:12: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(filepath=self.config.model_weights,model=self.base_mod)


In [27]:
import sys